<a href="https://colab.research.google.com/github/sunil-karki/Google3QLA-col/blob/master/Google3QLA_COLAB_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files
uploaded = files.upload()

Saving sample_submission.csv to sample_submission.csv
Saving test.csv to test.csv
Saving train.csv to train (1).csv


In [0]:
import io
import pandas as pd
X_train = pd.read_csv(io.BytesIO(uploaded['train.csv']))
# Dataset is now stored in a Pandas Dataframe

#X_train = pd.read_csv("../input/google-quest-challenge/train.csv")
print( len(X_train.columns), ' ', len(X_train))

41   6079


In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [0]:
!pip install kaggle

In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sunilkarki520","key":"8368d7fdc07a580b54aeb9a6df65cda4"}'}

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle competitions download -c google-quest-challenge 

100% 4.50M/4.50M [00:00<00:00, 18.9MB/s]

  0% 0.00/115k [00:00<?, ?B/s]
100% 115k/115k [00:00<00:00, 116MB/s]
  0% 0.00/954k [00:00<?, ?B/s]
100% 954k/954k [00:00<00:00, 63.5MB/s]


In [4]:
!ls

kaggle.json  sample_data  sample_submission.csv  test.csv  train.csv.zip


In [0]:
import pandas as pd
#df = pd.read_csv('filename.zip', compression='zip', header=0, sep=',', quotechar='"')
X_train = pd.read_csv('train.csv.zip', compression='zip') 

#X_train[['qa_id','question_title','question_body','question_user_name','question_user_page','answer','answer_user_name','answer_user_page','url',
#        'category','host']]

In [0]:
test_file = pd.read_csv('test.csv')

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

In [8]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [0]:
#Imports
from keras.models import Sequential, Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Activation, Dense, Permute, Dropout, add, dot, concatenate, LSTM

from keras import optimizers

from sklearn.model_selection import train_test_split

In [0]:
#print(vocab)
ylabels = ['question_asker_intent_understanding','question_body_critical','question_conversational','question_expect_short_answer',
'question_fact_seeking','question_has_commonly_accepted_answer','question_interestingness_others','question_interestingness_self',
'question_multi_intent','question_not_really_a_question','question_opinion_seeking','question_type_choice','question_type_compare',
'question_type_consequence','question_type_definition','question_type_entity','question_type_instructions','question_type_procedure',
'question_type_reason_explanation','question_type_spelling','question_well_written','answer_helpful','answer_level_of_information',
'answer_plausible','answer_relevance','answer_satisfaction','answer_type_instructions','answer_type_procedure','answer_type_reason_explanation',
'answer_well_written']

y = X_train[ylabels].astype(np.int64)



In [0]:
X_train = X_train[['qa_id','question_title','question_body','question_user_name','question_user_page','answer','answer_user_name','answer_user_page','url',
        'category','host']]
        

In [12]:
X_train[['qa_id','question_title','question_body','answer','category']]

,qa_id,question_title,question_body,answer,category
0,0,What am I losing when using extension tubes in...,After playing around with macro photography on...,"I just got extension tubes, so here's the skin...",LIFE_ARTS
1,1,What is the distinction between a city and a s...,I am trying to understand what kinds of places...,It might be helpful to look into the definitio...,CULTURE
2,2,Maximum protusion length for through-hole comp...,I'm working on a PCB that has through-hole com...,Do you even need grooves? We make several pro...,SCIENCE
3,3,Can an affidavit be used in Beit Din?,"An affidavit, from what i understand, is basic...","Sending an ""affidavit"" it is a dispute between...",CULTURE
4,5,How do you make a binary image in Photoshop?,I am trying to make a binary image. I want mor...,Check out Image Trace in Adobe Illustrator. \n...,LIFE_ARTS
...,...,...,...,...,...
6074,9642,Using a ski helmet for winter biking,I am curious if anyone uses a skiing helmet fo...,If you're thinking about wearing a ski helmet ...,CULTURE
6075,9643,Adjustment to road bike brakes for high grade ...,I have a road bike with a front brake that wea...,\nYou can replace the pads (as stated elsewher...,CULTURE
6076,9645,Suppress 'file truncated' messages when using ...,I'm tailing a log file using tail -f messages....,Maybe help if can be fixes origin of this erro...,TECHNOLOGY
6077,9646,When should a supervisor be a co-author?,What are people's views on this? To be specif...,"As a non-mathematician, I am somewhat mystifie...",SCIENCE


In [13]:
test_file.columns

Index(['qa_id', 'question_title', 'question_body', 'question_user_name',
       'question_user_page', 'answer', 'answer_user_name', 'answer_user_page',
       'url', 'category', 'host'],
      dtype='object')

In [14]:
test_file.iloc[0:5]

,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [0]:
all_data = pd.concat([X_train, test_file])

In [16]:
all_data.shape

(6555, 11)

In [17]:
all_data.iloc[6076: 6082]

,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
6076,9645,Suppress 'file truncated' messages when using ...,I'm tailing a log file using tail -f messages....,Maneating Koala,https://unix.stackexchange.com/users/60445,Maybe help if can be fixes origin of this erro...,BG Bruno,https://unix.stackexchange.com/users/68208,http://unix.stackexchange.com/questions/169054...,TECHNOLOGY,unix.stackexchange.com
6077,9646,When should a supervisor be a co-author?,What are people's views on this? To be specif...,MrB,https://mathoverflow.net/users/2189,"As a non-mathematician, I am somewhat mystifie...",angela,https://mathoverflow.net/users/4267,http://mathoverflow.net/questions/57337,SCIENCE,mathoverflow.net
6078,9647,Why are there so many different types of screw...,Newbie question.\n\nWhy is it that there's a b...,Doug T.,https://diy.stackexchange.com/users/321,"First, I really like Eric's answer for practic...",Scivitri,https://diy.stackexchange.com/users/113,http://diy.stackexchange.com/questions/2701/wh...,LIFE_ARTS,diy.stackexchange.com
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com


In [0]:
#X_train['question_body'][0]
##  https://www.figure-eight.com/data-for-everyone/

In [0]:
# 'question_title','question_body','category' to be added together here
def add_col(X):
    X['question_body'] = X['question_body'] + ' ' + X['question_title'] + ' ' + X['category']
    del X['question_title']
    del X['category']
    #X_train['category']
    return X

all_data = add_col(all_data)
X_train = add_col(X_train)

In [20]:
X_train.columns

Index(['qa_id', 'question_body', 'question_user_name', 'question_user_page',
       'answer', 'answer_user_name', 'answer_user_page', 'url', 'host'],
      dtype='object')

In [21]:
import nltk
#Resource stopwords and wordnet not found
#needed to do this below in colab
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
#import nltk
import re

stopword = nltk.corpus.stopwords.words('english')

lm = nltk.WordNetLemmatizer()
st = nltk.PorterStemmer()

class NLP_preprocess:
    def tokenize(self, text):
        tokens = re.split('\W+', text)
        return tokens

    def remove_stopwords(self, tokenized_text):
        text = [word for word in tokenized_text if word not in stopword]
        return text
    
    def lemmatizing(self, text):
        text = [lm.lemmatize(word) for word in text]
        return text
    
    def stemming(self, text):
        text = [st.stem(word) for word in text]
        return text

    def process_column(self, col):
        col = col.apply(lambda x: self.tokenize(x.lower()))
        col = col.apply(lambda x: self.remove_stopwords(x))
        col = col.apply(lambda x: self.lemmatizing(x))
        return col


In [0]:
nlp = NLP_preprocess()

#Contains column which are text. Needs to be converted to Token
list_col = ['answer', 'question_body']

for col in list_col:
    X_train[col] = nlp.process_column(X_train[col])

for col in list_col:
    all_data[col] = nlp.process_column(all_data[col])

In [0]:
class NLP_columns:
    def join_col(self, list_col):
        new_col = X_train[list_col[0]]
    
        for col in list_col[1:]:
            new_col += X_train[col]
        
        return new_col

    #Removing unnecessary elements from the processes joined Tokenized column
    def remove_values_from_list(self, the_list, val):
        return [value for value in the_list if value != val]
    
    def remove_values_from_column(self, col, val):
        return col.apply(lambda row: self.remove_values_from_list(row, val))


In [25]:
'''
def process_vocab():
    vocab = set()
    for a in X_train['answer']:
        vocab = vocab.union(set(a))

    for a in X_train['question_body']:
        vocab = vocab.union(set(a))
    
    vocab_len = len(vocab)
    #print(vocab_len)      #49474
    return vocab_len
    
vocab_len = process_vocab()
print(vocab_len)
'''

"\ndef process_vocab():\n    vocab = set()\n    for a in X_train['answer']:\n        vocab = vocab.union(set(a))\n\n    for a in X_train['question_body']:\n        vocab = vocab.union(set(a))\n    \n    vocab_len = len(vocab)\n    #print(vocab_len)      #49474\n    return vocab_len\n    \nvocab_len = process_vocab()\nprint(vocab_len)\n"

In [26]:
'''
#print(vocab)
ylabels = ['question_asker_intent_understanding','question_body_critical','question_conversational','question_expect_short_answer',
'question_fact_seeking','question_has_commonly_accepted_answer','question_interestingness_others','question_interestingness_self',
'question_multi_intent','question_not_really_a_question','question_opinion_seeking','question_type_choice','question_type_compare',
'question_type_consequence','question_type_definition','question_type_entity','question_type_instructions','question_type_procedure',
'question_type_reason_explanation','question_type_spelling','question_well_written','answer_helpful','answer_level_of_information',
'answer_plausible','answer_relevance','answer_satisfaction','answer_type_instructions','answer_type_procedure','answer_type_reason_explanation',
'answer_well_written']

y = X_train[ylabels].astype(np.int64)

'''

"\n#print(vocab)\nylabels = ['question_asker_intent_understanding','question_body_critical','question_conversational','question_expect_short_answer',\n'question_fact_seeking','question_has_commonly_accepted_answer','question_interestingness_others','question_interestingness_self',\n'question_multi_intent','question_not_really_a_question','question_opinion_seeking','question_type_choice','question_type_compare',\n'question_type_consequence','question_type_definition','question_type_entity','question_type_instructions','question_type_procedure',\n'question_type_reason_explanation','question_type_spelling','question_well_written','answer_helpful','answer_level_of_information',\n'answer_plausible','answer_relevance','answer_satisfaction','answer_type_instructions','answer_type_procedure','answer_type_reason_explanation',\n'answer_well_written']\n\ny = X_train[ylabels].astype(np.int64)\n\n"

In [27]:
X_train.columns

Index(['qa_id', 'question_body', 'question_user_name', 'question_user_page',
       'answer', 'answer_user_name', 'answer_user_page', 'url', 'host'],
      dtype='object')

In [28]:
#Delete unwanted columns from X_train
#del X_train['question_asker_intent_understanding']
#X_train.drop(['question_asker_intent_understanding', 'question_body_critical'], axis=1, inplace=True)
#X_train.drop(ylabels, axis=1, inplace=True)

KeyError: ignored

In [29]:
X_train.columns     #Will need only  'answer' 'question_body'

Index(['qa_id', 'question_body', 'question_user_name', 'question_user_page',
       'answer', 'answer_user_name', 'answer_user_page', 'url', 'host'],
      dtype='object')

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y, test_size=0.33, random_state=np.random, shuffle=True)

In [31]:
print(X_test.shape)

(2007, 9)


In [33]:
'''
class NLP_before_model:
    tokenizer = 0
    max_question_body_len = 0
    max_answers_seq_len = 0
    train_question_body = []
    train_answers = []
    
    x_answer = 0
    x_question = 0
    
    vocab = set()
    vocab_len = 0
    
    def __init__(self, x_answer, x_question):
        self.x_answer = x_answer
        self.x_question = x_question
        
        self.process_vocab()
        self.tokenizer = Tokenizer(filters = [])
        self.tokenizer.fit_on_texts(self.vocab)
        
        #self.process_for_model()
        
    def process_vocab(self):
        #vocab = set()
        for a in self.x_answer:
            self.vocab = self.vocab.union(set(a))

        for a in self.x_question:
            self.vocab = self.vocab.union(set(a))

        self.vocab_len = len(self.vocab)
        #print(vocab_len)      #49474
        #return self.vocab_len
        
    def process_for_model(self):
        for a in self.x_answer:
            self.train_answers.append(a)

        for a in self.x_question:
            self.train_question_body.append(a)
            
        a, b = self.process_vectorize(self.train_answers, self.train_question_body)
        return a, b
                    
    def process_vectorize(self, train_answers, train_question_body):
        
        train_answers_seq = self.tokenizer.texts_to_sequences(train_answers)
        train_question_body_seq = self.tokenizer.texts_to_sequences(train_question_body)

        self.max_question_body_len = max([len(data) for data in train_question_body_seq])        
        self.max_answers_seq_len = max([len(data) for data in train_answers_seq])
        
        input_answer = pad_sequences(train_answers_seq, maxlen = self.max_answers_seq_len)
        input_question = pad_sequences(train_question_body_seq, maxlen = self.max_question_body_len)

        return input_answer, input_question

'''

'\nclass NLP_before_model:\n    tokenizer = 0\n    max_question_body_len = 0\n    max_answers_seq_len = 0\n    train_question_body = []\n    train_answers = []\n    \n    x_answer = 0\n    x_question = 0\n    \n    vocab = set()\n    vocab_len = 0\n    \n    def __init__(self, x_answer, x_question):\n        self.x_answer = x_answer\n        self.x_question = x_question\n        \n        self.process_vocab()\n        self.tokenizer = Tokenizer(filters = [])\n        self.tokenizer.fit_on_texts(self.vocab)\n        \n        #self.process_for_model()\n        \n    def process_vocab(self):\n        #vocab = set()\n        for a in self.x_answer:\n            self.vocab = self.vocab.union(set(a))\n\n        for a in self.x_question:\n            self.vocab = self.vocab.union(set(a))\n\n        self.vocab_len = len(self.vocab)\n        #print(vocab_len)      #49474\n        #return self.vocab_len\n        \n    def process_for_model(self):\n        for a in self.x_answer:\n            se

In [0]:
class NLP_before_model:
    tokenizer = 0
    max_question_body_len = 0
    max_answers_seq_len = 0
    train_question_body = []
    train_answers = []

    train_answers_seq = 0
    train_question_body_seq = 0

    max_question_len = 0
    max_answer_len = 0
    
    x_answer = 0
    x_question = 0
    
    vocab = set()
    vocab_len = 0
    
    def __init__(self, x_answer, x_question):
        self.x_answer = x_answer
        self.x_question = x_question
        
        self.process_vocab()
        self.tokenizer = Tokenizer(filters = [])
        self.tokenizer.fit_on_texts(self.vocab)
        
        #self.process_for_model()
        
    def process_vocab(self):
        #vocab = set()
        for a in self.x_answer:
            self.vocab = self.vocab.union(set(a))

        for a in self.x_question:
            self.vocab = self.vocab.union(set(a))

        self.vocab_len = len(self.vocab)
        #print(vocab_len)      #49474
        #return self.vocab_len

        self.max_question_len = max([len(data) for data in self.x_question])  #all_data = train + test
        self.max_answer_len =    max([len(data) for data in self.x_answer])

    def tokenize_wordindex(self):
        print(self.tokenizer.word_index)


    def process_for_model(self):
        for a in self.x_answer:
            self.train_answers.append(a)

        for a in self.x_question:
            self.train_question_body.append(a)


    def vectorize_stories(self, data_answer, data_question):
        word_index = self.tokenizer.word_index
        max_answer_len = self.max_answer_len
        max_question_len = self.max_question_len
        #vectorized stories:
        X = []
        #vectorized questions:
        Xq = []
        #vectorized answers:
        Y = []
        '''
        for a in data_answer:
            #Getting indexes for each word in the story
            x = [word_index[word.lower()] for word in a]
            X.append(x)
        
        for a in data_question:
            #Getting indexes for each word in the story
            x = [word_index[word.lower()] for word in a]
            Xq.append(x)
        '''
        train_answers_seq = self.tokenizer.texts_to_sequences(data_answer)
        train_question_body_seq = self.tokenizer.texts_to_sequences(data_question)
            
        #Now we have to pad these sequences:
        return(pad_sequences(train_answers_seq,maxlen=max_answer_len), 
               pad_sequences(train_question_body_seq, maxlen=max_question_len))    
   



In [0]:
#nlpbefore_mdl = NLP_before_model(X_train['answer'], X_train['question_body'])   #Both train and test should be there
nlpbefore_mdl = NLP_before_model(all_data['answer'], all_data['question_body'])

In [36]:
nlpbefore_mdl.tokenize_wordindex()

{'mu_2': 1, 'imagination': 2, 'showpdf': 3, 'scr': 4, 'inappropriately': 5, 'ubnd': 6, 'righttoleft': 7, '2016': 8, 'spooky': 9, '608': 10, 'analyzes': 11, 'um': 12, 'korban': 13, 'wireframes': 14, 'goody': 15, 'scrub': 16, 'doxylover': 17, 'updated_at': 18, '7862272': 19, 'dag': 20, 'fission': 21, '920': 22, 'qam': 23, 'renderto': 24, 'bge': 25, 'invalidated': 26, 'tllw': 27, 'hh': 28, 'rvert_': 29, 'pst_time': 30, 'stringstream': 31, 'trty': 32, 'fish': 33, 'pdip': 34, '286256': 35, 'imperative': 36, '864770': 37, 'taliban': 38, 'emptying': 39, 'involve': 40, 'multicollision': 41, 'reinvestment': 42, '1da43f': 43, 'cairo_print': 44, 'shb': 45, 'hood': 46, 'unregistered': 47, '107873947602648971741': 48, 'warrior': 49, 'geq0': 50, 'amazed': 51, 'owopiccob3xu9wa0nmb1z2xrnwyixgbhnmioqnchr3cfj2bkogckasfkayowrciujxd0vmcjogck0xck0xcjogc9lqcjoqfjkqcjowoxqhelrhjuahj9ahjjkqcjkgc7v2csvwfjkqcjoqfjkqcjkgc7kcckaclxqhelrhjuicii4yzhrhjgwyzhrhjoq3cylmzfv2yhxgclj3xyr3c9ahjjkqcjkqcksxzzxwz9lqcjkqckayo

In [37]:
#Vectorize trrain data
'''train_input_answer, train_input_question = nlpbefore_mdl.process_for_model()   #To vectorize only
train_input_answer'''

'train_input_answer, train_input_question = nlpbefore_mdl.process_for_model()   #To vectorize only\ntrain_input_answer'

In [38]:
#Vectorize trrain data
train_input_answer, train_input_question = nlpbefore_mdl.vectorize_stories(X_train['answer'], X_train['question_body'])   #To vectorize only
train_input_answer

array([[    0,     0,     0, ..., 33781,  5975, 21398],
       [    0,     0,     0, ..., 47219, 38628, 24457],
       [    0,     0,     0, ...,   558, 39865, 19689],
       ...,
       [    0,     0,     0, ..., 47967,  6730, 36914],
       [    0,     0,     0, ..., 38081, 20715, 38503],
       [    0,     0,     0, ..., 33337, 28132, 20180]], dtype=int32)

In [39]:
max_answers_seq_len = nlpbefore_mdl.max_answers_seq_len
max_question_body_len = nlpbefore_mdl.max_question_body_len
vocab_len = nlpbefore_mdl.vocab_len
print(max_answers_seq_len)
print(max_question_body_len)
print("vocab_len : ", vocab_len)
max_question_len = nlpbefore_mdl.max_question_len
max_answer_len = nlpbefore_mdl.max_answer_len
print(max_question_len)
print(max_answer_len)

0
0
vocab_len :  53173
1832
2090


In [40]:
#Vectorize test data
test_input_answer, test_input_question = nlpbefore_mdl.vectorize_stories(X_test['answer'], X_test['question_body'])
test_input_answer

array([[    0,     0,     0, ..., 10504, 15467, 44356],
       [    0,     0,     0, ...,  5629, 19838,  8648],
       [    0,     0,     0, ...,  1856, 47967,  7536],
       ...,
       [    0,     0,     0, ..., 36381, 23470, 49710],
       [    0,     0,     0, ...,  3904, 52269,  4557],
       [    0,     0,     0, ..., 13033, 49710, 25351]], dtype=int32)

In [41]:
'''
#Vectorize test data
test_input_answer, test_input_question = nlpbefore_mdl.process_vectorize(X_test['answer'], X_test['question_body'])
test_input_answer
'''

"\n#Vectorize test data\ntest_input_answer, test_input_question = nlpbefore_mdl.process_vectorize(X_test['answer'], X_test['question_body'])\ntest_input_answer\n"

In [0]:
from keras.optimizers import Adam
#from keras.losses import huber_loss

In [0]:
class NLP_model():
    max_answers_seq_len = 0
    max_question_body_len = 0
    vocab_len = 0
    
    model = 0
    history = 0
    a = 0
    
    
    def __init__(self, max_answers_seq_len, max_question_body_len, vocab_len):
        self.max_answers_seq_len = max_answers_seq_len
        self.max_question_body_len = max_question_body_len
        self.vocab_len = vocab_len
    
    def create_model(self):
        input_sequence = Input((self.max_answers_seq_len,)) #As we dont know batch size yet
        question = Input((self.max_question_body_len,))

        #Create input encoder M
        input_encoder_m = Sequential()
        input_encoder_m.add(Embedding(input_dim = self.vocab_len,output_dim = 64)) #From paper
        input_encoder_m.add(Dropout(0.3))

        #Create input encoder C:
        input_encoder_c = Sequential()
        input_encoder_c.add(Embedding(input_dim = self.vocab_len,output_dim = self.max_question_body_len)) #From paper
        input_encoder_c.add(Dropout(0.3))

        #Create question encoder:
        question_encoder = Sequential()
        question_encoder.add(Embedding(input_dim = self.vocab_len,output_dim = 64,input_length = self.max_question_body_len)) #From paper
        question_encoder.add(Dropout(0.3))

        input_encoded_m = input_encoder_m(input_sequence)
        input_encoded_c = input_encoder_c(input_sequence)
        question_encoded = question_encoder(question)

        match = dot([input_encoded_m,question_encoded], axes = (2,2))
        match = Activation('softmax')(match)

        response = add([match,input_encoded_c])
        response = Permute((2,1))(response) #Permute Layer: permutes dimensions of input

        answer = concatenate([response, question_encoded])

        # Reduce the answer tensor with a RNN (LSTM)
        answer = LSTM(32)(answer)

        #Regularization with dropout:
        answer = Dropout(0.5)(answer)
        #Output layer:
        answer = Dense(30)(answer) #Output shape: (Samples, Vocab_size) #Yes or no and all 0s

        #Now we need to output a probability distribution for the vocab, using softmax:
        answer = Activation('softmax')(answer)

        #Now we build the final model:
        self.model = Model([input_sequence,question], answer)

    def compile_model(self):
        #sgd = optimizers.SGD(lr=0.5, decay=1e-5, momentum=0.7, nesterov=True)  #0.01

        #self.model.compile(loss='poisson', optimizer=sgd, metrics = ['accuracy'])
        
        #model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
        #model.compile(optimizer='rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])
        #model.compile(optimizer='rmsprop', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
        # model.compile(optimizer='rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])
        #Categorical instead of binary cross entropy as because of the way we are training
        #we could actually see any of the words from the vocab as output
        #however, we should only see yes or no

        ## self.model.compile(optimizer='rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy']) #acc 6%
        #self.model.compile(optimizer='rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy']) #accuracy around 75%
        self.model.compile(optimizer=Adam(lr=0.05), loss = 'binary_crossentropy', metrics = ['accuracy'])
        
    def summary_model(self):
        self.model.summary()
        
    def fit_model(self, input_labels, input_answer, input_question):
        #self.a = input_labels.astype(np.int64)
        #input_train == answeer  answert train == label // label left tot do
        #  history = model.fit([inputs_train,questions_train],a, batch_size = 32, epochs = 1000, validation_data = ([inputs_test,questions_test],answers_test))
        
        #print(type(self.a))
        #self.history = self.model.fit([input_answer,input_question], a, batch_size = 50, epochs = 1000)
        
        #self.history = self.model.fit([input_answer,input_question], input_labels, batch_size = 75, epochs = 1500)

        try:
          self.history = self.model.fit([input_answer,input_question], input_labels, batch_size = 75, epochs = 1000)
        except KeyboardInterrupt:
          self.save_model();
        #xxxxx
        
    def eval_model(self, answer, question):
        pred_results = self.model.predict(([answer,question]))
        
    def save_model(self):
        filename = 'google3qla.h5'
        self.model.save(filename)
        print("Model saved in file : ", filename)
        

In [46]:
nlpmodel = NLP_model(max_answer_len, max_question_len, vocab_len)
nlpmodel.create_model()
nlpmodel.compile_model()
nlpmodel.summary_model()





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 2090)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1832)         0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       multiple             3403072     input_1[0][0]                    
___________________________________________________________________________

In [47]:
nlpmodel.fit_model(y_train, train_input_answer, train_input_question)




Epoch 1/1000





4072/4072 [==============================] - 931s 229ms/step - loss: 0.7048 - acc: 0.7633
Epoch 2/1000
4072/4072 [==============================] - 773s 190ms/step - loss: 0.6845 - acc: 0.7633
Epoch 3/1000
4072/4072 [==============================] - 765s 188ms/step - loss: 0.6831 - acc: 0.7633
Epoch 4/1000
4072/4072 [==============================] - 765s 188ms/step - loss: 0.6814 - acc: 0.7633
Epoch 5/1000
4072/4072 [==============================] - 768s 189ms/step - loss: 0.6798 - acc: 0.7633
Epoch 6/1000
4072/4072 [==============================] - 770s 189ms/step - loss: 0.6779 - acc: 0.7633
Epoch 7/1000
 300/4072 [=>............................] - ETA: 12:00 - loss: 0.6862 - acc: 0.7583Model saved in file :  google3qla.h5


In [48]:
!ls

google3qla.h5  sample_data	      test.csv
kaggle.json    sample_submission.csv  train.csv.zip


In [0]:
#print(train_input_answer.shape)
nlpmodel.save_model()

In [0]:
from keras.models import load_model
ld_model = load_model('google3qla.h5')

In [50]:
print(train_input_answer.shape[1] - test_input_answer.shape[1])
print(train_input_question.shape[1] - test_input_question.shape[1])
print(train_input_answer.shape)
print(test_input_answer.shape)

0
0
(4072, 2090)
(2007, 2090)


In [51]:
score = ld_model.evaluate([test_input_answer, test_input_question], y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.6861208888150354
Test accuracy: 0.760720811379627


In [60]:
test_file_1 = all_data.iloc[6079:]
test_file_1.shape

(476, 9)

In [61]:
#Vectorize test data
testfile_input_answer, testfile_input_question = nlpbefore_mdl.vectorize_stories(test_file_1['answer'], test_file_1['question_body'])
testfile_input_answer

array([[    0,     0,     0, ..., 51956, 36582,  8759],
       [    0,     0,     0, ..., 47967, 34367, 47967],
       [    0,     0,     0, ...,  3774,  9406,  9983],
       ...,
       [    0,     0,     0, ...,  9756, 16478, 48954],
       [    0,     0,     0, ..., 50999, 52551, 52189],
       [    0,     0,     0, ..., 43240, 52720,  2177]], dtype=int32)

In [67]:
prediction = ld_model.predict([testfile_input_answer, testfile_input_question])
prediction.shape

(476, 30)

In [70]:
type(prediction)

numpy.ndarray

In [0]:
np.set_printoptions(precision=2,
                       threshold=10000,
                       linewidth=150)

In [65]:
len(ylabels)

30

In [76]:
prediction

array([[0.09, 0.01, 0.  , ..., 0.  , 0.05, 0.06],
       [0.05, 0.  , 0.  , ..., 0.  , 0.02, 0.04],
       [0.09, 0.01, 0.  , ..., 0.  , 0.05, 0.06],
       ...,
       [0.06, 0.  , 0.  , ..., 0.  , 0.02, 0.04],
       [0.09, 0.01, 0.  , ..., 0.  , 0.05, 0.06],
       [0.09, 0.01, 0.  , ..., 0.  , 0.05, 0.06]], dtype=float32)

In [77]:
!ls

google3qla.h5  sample_data	      test.csv
kaggle.json    sample_submission.csv  train.csv.zip


In [78]:
submission = pd.read_csv('sample_submission.csv')
submission

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,question_not_really_a_question,question_opinion_seeking,question_type_choice,question_type_compare,question_type_consequence,question_type_definition,question_type_entity,question_type_instructions,question_type_procedure,question_type_reason_explanation,question_type_spelling,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308
1,46,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448
2,70,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673
3,132,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401
4,200,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,9569,0.99159,0.99159,0.99159,0.99159,0.99159,0.99159,0.99159,0.99159,0.99159,0.99159,0.99159,0.99159,0.99159,0.99159,0.99159,0.99159,0.99159,0.99159,0.99159,0.99159,0.99159,0.99159,0.99159,0.99159,0.99159,0.99159,0.99159,0.99159,0.99159,0.99159
472,9590,0.99355,0.99355,0.99355,0.99355,0.99355,0.99355,0.99355,0.99355,0.99355,0.99355,0.99355,0.99355,0.99355,0.99355,0.99355,0.99355,0.99355,0.99355,0.99355,0.99355,0.99355,0.99355,0.99355,0.99355,0.99355,0.99355,0.99355,0.99355,0.99355,0.99355
473,9597,0.99467,0.99467,0.99467,0.99467,0.99467,0.99467,0.99467,0.99467,0.99467,0.99467,0.99467,0.99467,0.99467,0.99467,0.99467,0.99467,0.99467,0.99467,0.99467,0.99467,0.99467,0.99467,0.99467,0.99467,0.99467,0.99467,0.99467,0.99467,0.99467,0.99467
474,9623,0.99720,0.99720,0.99720,0.99720,0.99720,0.99720,0.99720,0.99720,0.99720,0.99720,0.99720,0.99720,0.99720,0.99720,0.99720,0.99720,0.99720,0.99720,0.99720,0.99720,0.99720,0.99720,0.99720,0.99720,0.99720,0.99720,0.99720,0.99720,0.99720,0.99720


In [84]:
sub = pd.DataFrame(data=prediction, columns=ylabels)
sub

,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,question_not_really_a_question,question_opinion_seeking,question_type_choice,question_type_compare,question_type_consequence,question_type_definition,question_type_entity,question_type_instructions,question_type_procedure,question_type_reason_explanation,question_type_spelling,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,0.087118,0.013962,0.003616,0.066872,0.087312,0.083712,0.003094,0.006025,0.015980,0.000087,0.023890,0.031243,0.002960,0.000480,0.001810,0.003882,0.014213,0.003082,0.033787,0.000135,0.057061,0.086493,0.003519,0.101996,0.111100,0.026405,0.012560,0.001789,0.052344,0.063472
1,0.054116,0.004432,0.000094,0.062980,0.075844,0.119817,0.001359,0.004438,0.004830,0.000004,0.022640,0.008631,0.000165,0.000017,0.000021,0.000362,0.089282,0.003488,0.019896,0.000005,0.024383,0.096390,0.000607,0.121490,0.124025,0.023026,0.073322,0.002517,0.020412,0.041406
2,0.087118,0.013962,0.003616,0.066872,0.087312,0.083712,0.003094,0.006025,0.015980,0.000087,0.023890,0.031243,0.002960,0.000480,0.001810,0.003882,0.014213,0.003082,0.033787,0.000135,0.057061,0.086493,0.003519,0.101996,0.111100,0.026405,0.012560,0.001789,0.052344,0.063472
3,0.078339,0.012283,0.002836,0.065890,0.086762,0.088896,0.002718,0.005550,0.016244,0.000272,0.024559,0.030107,0.002810,0.000859,0.001646,0.002826,0.025292,0.003488,0.031793,0.000363,0.046070,0.087940,0.003158,0.104615,0.113456,0.027249,0.022956,0.002351,0.049684,0.058990
4,0.087118,0.013962,0.003616,0.066872,0.087312,0.083712,0.003094,0.006025,0.015980,0.000087,0.023890,0.031243,0.002960,0.000480,0.001810,0.003882,0.014213,0.003082,0.033787,0.000135,0.057061,0.086493,0.003519,0.101996,0.111100,0.026405,0.012560,0.001789,0.052344,0.063472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,0.054116,0.004432,0.000094,0.062980,0.075844,0.119817,0.001359,0.004438,0.004830,0.000004,0.022640,0.008631,0.000165,0.000017,0.000021,0.000362,0.089282,0.003488,0.019896,0.000005,0.024383,0.096390,0.000607,0.121490,0.124025,0.023026,0.073322,0.002517,0.020412,0.041406
472,0.055584,0.004597,0.000120,0.062103,0.076124,0.117663,0.001410,0.004758,0.005394,0.000006,0.023328,0.009155,0.000197,0.000022,0.000029,0.000436,0.086848,0.003585,0.019929,0.000006,0.025559,0.096331,0.000684,0.120912,0.124251,0.023230,0.071374,0.002547,0.020990,0.042830
473,0.055584,0.004597,0.000120,0.062103,0.076124,0.117663,0.001410,0.004758,0.005394,0.000006,0.023328,0.009155,0.000197,0.000022,0.000029,0.000436,0.086848,0.003585,0.019929,0.000006,0.025559,0.096331,0.000684,0.120912,0.124251,0.023230,0.071374,0.002547,0.020990,0.042830
474,0.087118,0.013962,0.003616,0.066872,0.087312,0.083712,0.003094,0.006025,0.015980,0.000087,0.023890,0.031243,0.002960,0.000480,0.001810,0.003882,0.014213,0.003082,0.033787,0.000135,0.057061,0.086493,0.003519,0.101996,0.111100,0.026405,0.012560,0.001789,0.052344,0.063472


In [89]:
#submit = pd.DataFrame({'qa_id':test_file['qa_id'],'Survived':predictions})
submit = pd.concat([test_file_1['qa_id'], sub], axis=1)
submit

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,question_not_really_a_question,question_opinion_seeking,question_type_choice,question_type_compare,question_type_consequence,question_type_definition,question_type_entity,question_type_instructions,question_type_procedure,question_type_reason_explanation,question_type_spelling,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.087118,0.013962,0.003616,0.066872,0.087312,0.083712,0.003094,0.006025,0.015980,0.000087,0.023890,0.031243,0.002960,0.000480,0.001810,0.003882,0.014213,0.003082,0.033787,0.000135,0.057061,0.086493,0.003519,0.101996,0.111100,0.026405,0.012560,0.001789,0.052344,0.063472
1,46,0.054116,0.004432,0.000094,0.062980,0.075844,0.119817,0.001359,0.004438,0.004830,0.000004,0.022640,0.008631,0.000165,0.000017,0.000021,0.000362,0.089282,0.003488,0.019896,0.000005,0.024383,0.096390,0.000607,0.121490,0.124025,0.023026,0.073322,0.002517,0.020412,0.041406
2,70,0.087118,0.013962,0.003616,0.066872,0.087312,0.083712,0.003094,0.006025,0.015980,0.000087,0.023890,0.031243,0.002960,0.000480,0.001810,0.003882,0.014213,0.003082,0.033787,0.000135,0.057061,0.086493,0.003519,0.101996,0.111100,0.026405,0.012560,0.001789,0.052344,0.063472
3,132,0.078339,0.012283,0.002836,0.065890,0.086762,0.088896,0.002718,0.005550,0.016244,0.000272,0.024559,0.030107,0.002810,0.000859,0.001646,0.002826,0.025292,0.003488,0.031793,0.000363,0.046070,0.087940,0.003158,0.104615,0.113456,0.027249,0.022956,0.002351,0.049684,0.058990
4,200,0.087118,0.013962,0.003616,0.066872,0.087312,0.083712,0.003094,0.006025,0.015980,0.000087,0.023890,0.031243,0.002960,0.000480,0.001810,0.003882,0.014213,0.003082,0.033787,0.000135,0.057061,0.086493,0.003519,0.101996,0.111100,0.026405,0.012560,0.001789,0.052344,0.063472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,9569,0.054116,0.004432,0.000094,0.062980,0.075844,0.119817,0.001359,0.004438,0.004830,0.000004,0.022640,0.008631,0.000165,0.000017,0.000021,0.000362,0.089282,0.003488,0.019896,0.000005,0.024383,0.096390,0.000607,0.121490,0.124025,0.023026,0.073322,0.002517,0.020412,0.041406
472,9590,0.055584,0.004597,0.000120,0.062103,0.076124,0.117663,0.001410,0.004758,0.005394,0.000006,0.023328,0.009155,0.000197,0.000022,0.000029,0.000436,0.086848,0.003585,0.019929,0.000006,0.025559,0.096331,0.000684,0.120912,0.124251,0.023230,0.071374,0.002547,0.020990,0.042830
473,9597,0.055584,0.004597,0.000120,0.062103,0.076124,0.117663,0.001410,0.004758,0.005394,0.000006,0.023328,0.009155,0.000197,0.000022,0.000029,0.000436,0.086848,0.003585,0.019929,0.000006,0.025559,0.096331,0.000684,0.120912,0.124251,0.023230,0.071374,0.002547,0.020990,0.042830
474,9623,0.087118,0.013962,0.003616,0.066872,0.087312,0.083712,0.003094,0.006025,0.015980,0.000087,0.023890,0.031243,0.002960,0.000480,0.001810,0.003882,0.014213,0.003082,0.033787,0.000135,0.057061,0.086493,0.003519,0.101996,0.111100,0.026405,0.012560,0.001789,0.052344,0.063472


In [90]:
print(submit.shape)

(476, 31)


In [0]:
submit.to_csv('output_file', sep='\t')

In [92]:
!ls

google3qla.h5  output_file  sample_submission.csv  train.csv.zip
kaggle.json    sample_data  test.csv


In [0]:
###   Rough Work  ###